# A Crash Course in Accelerated Database Recovery

In [35]:
USE master;

SET NOCOUNT ON;

DROP DATABASE IF EXISTS AdrDb;

CREATE DATABASE AdrDb;
GO

DROP TABLE IF EXISTS AdrTest;

USE AdrDb;

CREATE TABLE AdrTest
(RowId INT IDENTITY(1,1)
,FirstName NVARCHAR(100)
,LastName NVARCHAR(100));

DECLARE @loopcount INT, 
@looplimit INT;

SELECT @loopcount = 1, @looplimit = 250000;

WHILE @loopcount <= @looplimit
BEGIN

    INSERT INTO AdrTest
    (FirstName
    ,LastName)
    VALUES
    (N'Terence',N'Butler'),
    (N'John',N'Osbourne'),
    (N'Anthony',N'Iommi'),
    (N'William',N'Ward');

    SELECT @loopcount += 1;

END

: Msg 1807, Level 16, State 3, Line 7
Could not obtain exclusive lock on database 'model'. Retry the operation later.

: Msg 1802, Level 16, State 4, Line 7
CREATE DATABASE failed. Some file names listed could not be created. Check related errors.

Query was canceled by user

: Query failed: The operation was canceled.

Query was canceled by user

Total execution time: 00:00:02.091

In [ ]:
USE master
BACKUP DATABASE AdrDb; 

In [37]:
USE master;

ALTER DATABASE AdrDb
SET SINGLE_USER WITH ROLLBACK IMMEDIATE

RESTORE DATABASE AdrDb FROM DISK=N'/var/opt/mssql/data/AdrDb.bak' WITH REPLACE, STATS = 5;

6 percent processed.

10 percent processed.

15 percent processed.

21 percent processed.

26 percent processed.

...

100 percent processed.

Processed 5896 pages for database 'AdrDb', file 'AdrDb' on file 1.

Processed 2 pages for database 'AdrDb', file 'AdrDb_log' on file 1.

RESTORE DATABASE successfully processed 5898 pages in 1.109 seconds (41.545 MB/sec).

Total execution time: 00:00:03.444

In [43]:
USE AdrDb;

SELECT name, is_accelerated_database_recovery_on
FROM sys.databases
WHERE name = N'AdrDb';

Commands completed successfully.

Total execution time: 00:00:00.018

name,is_accelerated_database_recovery_on
AdrDb,0


In [40]:
USE AdrDb;

DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

BEGIN TRANSACTION

DELETE FROM AdrTest
WHERE LastName = 'Osbourne';

SELECT Operation
,[Description]
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE Operation = 'LOP_DELETE_ROWS'
GROUP BY Operation, [Description];
GO

DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

ROLLBACK TRANSACTION;

SELECT Operation
,[Description]
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE [Description] = 'Compensation'
GROUP BY Operation, [Description];
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:13.163

Operation,Description,TranCount
LOP_DELETE_ROWS,,250000


Operation,Description,TranCount
LOP_INSERT_ROWS,COMPENSATION,250000


In [44]:
USE master;

ALTER DATABASE AdrDb
SET ACCELERATED_DATABASE_RECOVERY = ON;

SELECT name, is_accelerated_database_recovery_on
FROM sys.databases
WHERE name = N'AdrDb';

Commands completed successfully.

Total execution time: 00:00:00.130

name,is_accelerated_database_recovery_on
AdrDb,1


In [45]:
USE AdrDb;

DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

BEGIN TRANSACTION

DELETE FROM AdrTest
WHERE LastName = 'Osbourne';

SELECT Operation
,[Description]
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE Operation = 'LOP_DELETE_ROWS'
GROUP BY Operation, [Description];
GO

Commands completed successfully.

Total execution time: 00:00:19.531

Operation,Description,TranCount
LOP_DELETE_ROWS,,250000


In [46]:
DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

ROLLBACK TRANSACTION;

SELECT [Current LSN]
,[Transaction ID]
,[Transaction Name]
,Operation
,Context
,[Description]
,[Previous LSN]
,AllocUnitName
,[Page ID]
,[Slot ID]
,[Begin Time]
,[Database Name]
,[Number of Locks]
,[Lock Information]
,[New Split Page]
FROM fn_dblog(@maxlsn,NULL);

SELECT Operation
,[Description]
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE [Description] = 'Compensation'
GROUP BY Operation, [Description];
GO

Commands completed successfully.

Total execution time: 00:00:15.804

Current LSN,Transaction ID,Transaction Name,Operation,Context,Description,Previous LSN,AllocUnitName,Page ID,Slot ID,Begin Time,Database Name,Number of Locks,Lock Information,New Split Page
00000081:00003318:0163,0000:0003e98e,NULL,LOP_COMMIT_XACT,LCX_NULL,2021/02/16 21:27:11:733,00000081:00003228:00f7,NULL,NULL,NULL,NULL,none,NULL,NULL,NULL
00000081:00003318:0164,0000:0003e982,NULL,LOP_ABORT_XACT,LCX_CTR_ABORTED,2021/02/16 21:27:41:327,0000007d:00006d08:0001,NULL,NULL,NULL,NULL,none,NULL,NULL,NULL


Operation,Description,TranCount
